In [15]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression


In [34]:
import os

Dataset_Directory = '/home/jbenyam/threatdetection/dataset/csv' # Directory where the dataset is stored

# List all files in the directory and subdirectories
def get_all_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.relpath(os.path.join(root, file), start=directory))
    return csv_files

df_sets = get_all_csv_files(Dataset_Directory)
df_sets.sort() # Sort the files

print(f"Total CSV files found: {len(df_sets)}")
print("Files:", df_sets)

# Split the data set into training 80% and test 20%
train_set = df_sets[:int(len(df_sets)*0.8)]
test_set = df_sets[int(len(df_sets)*0.8):]

print(f"Number of training files: {len(train_set)}")
print(f"Number of test files: {len(test_set)}")

# If you need full paths for loading data later
# train_set_full_path = [os.path.join(Dataset_Directory, file) for file in train_set]
# test_set_full_path = [os.path.join(Dataset_Directory, file) for file in test_set]

#print("Train files full paths:", train_set_full_path)
#print("Test files full paths:", test_set_full_path)


Total CSV files found: 287
Files: ['Backdoor_Malware/Backdoor_Malware.pcap.csv', 'Benign_Final/BenignTraffic.pcap.csv', 'Benign_Final/BenignTraffic1.pcap.csv', 'Benign_Final/BenignTraffic2.pcap.csv', 'Benign_Final/BenignTraffic3.pcap.csv', 'BrowserHijacking/BrowserHijacking.pcap.csv', 'CommandInjection/CommandInjection.pcap.csv', 'DDoS-HTTP_Flood/DDoS-HTTP_Flood-.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood1.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood10.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood11.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood12.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood13.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood14.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood15.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood16.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood17.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood18.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood19.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood2.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Fl

In [31]:
print(train_set)
print(test_set)


['Backdoor_Malware/Backdoor_Malware.pcap.csv', 'Benign_Final/BenignTraffic.pcap.csv', 'Benign_Final/BenignTraffic1.pcap.csv', 'Benign_Final/BenignTraffic2.pcap.csv', 'Benign_Final/BenignTraffic3.pcap.csv', 'BrowserHijacking/BrowserHijacking.pcap.csv', 'CommandInjection/CommandInjection.pcap.csv', 'DDoS-HTTP_Flood/DDoS-HTTP_Flood-.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood1.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood10.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood11.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood12.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood13.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood14.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood15.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood16.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood17.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood18.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood19.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood2.pcap.csv', 'DDoS-ICMP_Flood/DDoS-ICMP_Flood20.pcap.csv', 'DDoS-ICMP_Flood/

In [35]:
# X_columns: list of columns to use as features
# Y_column: column to predict
# Model: model to use (logistic regression)

X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'



#### Scaling Features & Fitting the Scaler

In [36]:
# Importing feature scaler to help normalize feature values
# The scales of the features are different, so we need to normalize them to have mean 0 and standard deviation of 1

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler

# Fiting the scaler
for train_file in tqdm(train_set):
    # Construct the full path to the CSV file
    full_path = os.path.join(Dataset_Directory, train_file)
    
    # Read the CSV file
    df = pd.read_csv(full_path)
    
    # Check if the columns exist in the DataFrame
    if all(col in df.columns for col in X_columns):
        # Fit the scaler on the specified columns
        scaler.fit(df[X_columns])
        # Perform other operations as needed (e.g., transforming data)
    else:
        print(f"Warning: Not all specified columns are present in {train_file}")


  0%|          | 0/229 [00:00<?, ?it/s]

  1%|          | 2/229 [00:00<01:09,  3.27it/s]

  1%|▏         | 3/229 [00:01<01:23,  2.69it/s]

  2%|▏         | 5/229 [00:01<01:04,  3.50it/s]

  4%|▍         | 9/229 [00:01<00:32,  6.69it/s]

...

 91%|█████████▏| 209/229 [00:44<00:00, 20.17it/s]

 93%|█████████▎| 212/229 [00:44<00:01, 13.82it/s]

 95%|█████████▌| 218/229 [00:44<00:00, 18.23it/s]

 98%|█████████▊| 224/229 [00:44<00:00, 21.75it/s]

100%|██████████| 229/229 [00:45<00:00,  5.08it/s]

#### Model Classification

In [38]:
ML_models = [LogisticRegression(n_jobs=-1),] # List of ML models used for classification
ML_names = ['Logistic Regression',] # Model names for reference

# Training the models
for train_set in tqdm(train_set): # Loops through the training sets files
    d = pd.read_csv(Dataset_Directory + train_set) # Reads the dataset in Pandas
    d[X_columns] = scaler.transform(d[X_columns]) # Scales features in the dataset
    for model in (ML_models): 
        model.fit(d[X_columns], d[y_column]) # Trains the model on the scaled features and labels
    del d # Deletes the dataset to free up memory


  0%|          | 0/42 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/jbenyam/threatdetection/dataset/csvB'

#### Evaluating Model Performance

In [9]:
y_test = [] #  Initializes a list to store the true labels for the test sets.
preds = {i:[] for i in range(len(ML_models))} # Initializes a dictionary to store the predictions of the models for the test sets.
for test_set in tqdm(test_sets): # Loops through the test sets files
    d_test = pd.read_csv(DATASET_DIRECTORY + test_set) # Reads the dataset in Pandas
    d_test[X_columns] = scaler.transform(d_test[X_columns]) # Scales features in the dataset
    
    y_test += list(d_test[y_column].values) # Appends the true labels to the y_test list
    
    for i in range(len(ML_models)): # Loops through the models
        model = ML_models[i] # Selects the model
        y_pred = list(model.predict(d_test[X_columns])) # Predicts the labels of the test set
        preds[i] = preds[i] + y_pred # Appends the predictions to the dictionary


0it [00:00, ?it/s]


#### Calculating Metrics

In [11]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
for k,v in preds.items(): # Loops through the predictions
    y_pred = v # Retrieves the predictions for the model
    print(f"##### {ML_names[k]} (34 classes) #####") # Prints the model name 
    print('accuracy_score: ', accuracy_score(y_pred, y_test)) # Prints the accuracy score
    print('recall_score: ', recall_score(y_pred, y_test, average='macro')) # Prints the recall score
    print('precision_score: ', precision_score(y_pred, y_test, average='macro')) # Prints the precision score
    print('f1_score: ', f1_score(y_pred, y_test, average='macro')) # Prints the F1 score
    print()
    print()
    print()


##### Logistic Regression (34 classes) #####
accuracy_score:  nan
recall_score:  nan
precision_score:  nan
f1_score:  nan





#### Mapping Classes (7 classes)

##### A dictionary to map the original class labels to simplified class labels for a 7-class classification.

In [12]:
dict_11classes = {} # Initializes a dictionary to store the predictions of the models for the test sets.
dict_11classes['DDoS-ACK_Fragmentation'] = 'DDoS'
dict_11classes['DDoS-HTTP_Flood'] = 'DDoS'
dict_11classes['DDoS-ICMP_Flood'] = 'DDoS'
dict_11classes['DDoS-ICMP_Fragmentation'] = 'DDoS'
dict_11classes['DDoS-PSHACK_FLOOD'] = 'DDoS'
dict_11classes['DDoS-RSTFINFFLOOD'] = 'DDoS'
dict_11classes['DDoS-SlowLoris'] = 'DDoS'
dict_11classes['DDoS-SYN_Flood'] = 'DDoS'
dict_11classes['DDoS-SynonymousIP_Flood'] = 'DDoS'
dict_11classes['DDoS-TCP_Flood'] = 'DDoS'
dict_11classes['DDoS-UDP_Flood'] = 'DDoS'


### Outputing Metrics from Classification Report

In [14]:
from sklearn.metrics import classification_report
for k,v in preds.items(): # Maps the true labels to the 11 classes
    y_pred = v
    y_test_simple = [dict_11classes[y] for y in y_test] # Maps the true labels to the 11 labels
    y_pred_simple = [dict_11classes[y] for y in y_pred] # Maps the predicted labels to the 11 labels
    print(f"##### {ML_names[k]} (11 classes) #####")
    print(classification_report(y_test_simple, y_pred_simple))
    print()
    print()
    print()


##### Logistic Regression (11 classes) #####


ValueError: max() arg is an empty sequence